In [1]:
import pandas as pd
import networkx as nx
import umap
from tqdm import tqdm
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
#build ppi network
G = nx.from_pandas_edgelist(pd.read_csv('data/Gene_Gene.csv'), 'source', 'target')

In [3]:
#adj_mtx for embedding
A = nx.to_numpy_array(G)

#### 3D embedding

In [4]:
#pass adj_mtx to parametric UMAP
embedder = umap.ParametricUMAP(verbose=True, n_components=3)
embedding = embedder.fit_transform(A)

ParametricUMAP(optimizer=<keras.optimizer_v2.adam.Adam object at 0x7ff39a246a90>)
Wed Sep 14 02:19:23 2022 Construct fuzzy simplicial set
Wed Sep 14 02:19:24 2022 Finding Nearest Neighbors
Wed Sep 14 02:19:24 2022 Building RP forest with 12 trees
Wed Sep 14 02:20:35 2022 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	 5  /  14
	 6  /  14
	Stopping threshold met -- exiting after 6 iterations
Wed Sep 14 02:21:25 2022 Finished Nearest Neighbor Search
Wed Sep 14 02:21:27 2022 Construct embedding


2022-09-14 02:21:49.885369: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-14 02:21:49.889444: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/10
5655/5655 [==============================] - 1106s 195ms/step - loss: 0.3490
Epoch 2/10
5655/5655 [==============================] - 1102s 195ms/step - loss: 0.3169
Epoch 3/10
5655/5655 [==============================] - 1106s 196ms/step - loss: 0.3136
Epoch 4/10
5655/5655 [==============================] - 1103s 195ms/step - loss: 0.3122
Epoch 5/10
5655/5655 [==============================] - 1105s 195ms/step - loss: 0.3108
Epoch 6/10
5655/5655 [==============================] - 1110s 196ms/step - loss: 0.3099
Epoch 7/10
5655/5655 [==============================] - 1118s 198ms/step - loss: 0.3095
Epoch 8/10
5655/5655 [==============================] - 1107s 196ms/step - loss: 0.3089
Epoch 9/10
5655/5655 [==============================] - 1108s 196ms/step - loss: 0.3088
Epoch 10/10
577/577 [==============================] - 3s 5ms/step
Wed Sep 14 05:26:31 2022 Finished embedding


In [5]:
#coordinates in latent space using UMAP
df = []
c = 0
for node in G.nodes():
    df.append([node] + embedding[c].tolist())
    c += 1
cord_df = pd.DataFrame(df, columns=['GeneId', 'x', 'y', 'z'])              
cord_df.head(1)

,GeneId,x,y,z
0,PHYHIP,0.270208,-1.675647,0.444449


#### 2D embedding

In [20]:
cord_df = pd.read_csv('data/HPI_3D_ParametricUMAP.csv')
cord_df.head(1)

,node,x,y,z
0,PHYHIP,-2.155979,-0.209205,-0.311242


In [21]:
#embed to 2D
mapper = umap.UMAP().fit(cord_df[cord_df.columns[-3:]].values)
cord_df = cord_df.join(pd.DataFrame(mapper.embedding_.tolist(), columns=['x_2d', 'y_2d']))

In [22]:
#ppi in 2d space
g = figure(plot_width=600, plot_height=600, title='ppi in 2D embedding')
g.xgrid.visible = False
g.xaxis.visible = False
g.ygrid.visible = False
g.yaxis.visible = False

g.scatter(x='x_2d', y='y_2d', source=cord_df, size=2, color='grey')
show(g)

In [23]:
cord_df.to_csv('data/HPI_2D_ParametricUMAP.csv', index=False)